# 黑白图像转换

现在我们已经了解了多维索引，为什么不尝试使用二维索引将我们的图像转换为黑白的呢？

我们不是逐个通道操作所有像素，而是想要对所有像素进行操作并对通道取平均值。

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from numba import cuda
import numpy as np
import math

plt.rcParams["figure.figsize"] = (30, 4)

**1. 使用 matplotlib 加载我们的图像。**

In [ ]:
!wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/main/gpu-python-tutorial/images/numba.png
im = plt.imread("numba.png")
plt.imshow(im)

**2. 将图像移动到 GPU 并创建相同大小的输出数组。**

In [ ]:
gpu_im = cuda.to_device(im)
gpu_output = cuda.to_device(np.zeros_like(gpu_im))

**3. 设置我们的二维线程大小和块大小。** _提示：我们的 `threadsperblock` 的乘积仍应为 `128`。_

In [ ]:
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(gpu_im.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(gpu_im.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

**4. 编写我们的内核。**

In [ ]:
@cuda.jit
def black_white(im, output):
    # 使用我们的二维网格，我们可以在两个维度上获取索引位置
    x, y = cuda.grid(2)

    # 因为我们的网格略大于图像，所以图像外的任何内容都应该被忽略
    if x < im.shape[0] and y < im.shape[1]:

        # 计算 RGB 通道的平均值
        average = (im[x, y, 0] + im[x, y, 1] + im[x, y, 2]) / 3

        # 将所有输出 RGB 通道设置为平均值
        output[x, y, 0] = average
        output[x, y, 1] = average
        output[x, y, 2] = average

        # 保持透明度通道不变
        output[x, y, 3] = im[x, y, 3]

**5. 运行内核。**

In [ ]:
black_white[blockspergrid, threadsperblock](gpu_im, gpu_output)

**6. 将数据从 GPU 移回并绘制图像。**

In [ ]:
plt.imshow(gpu_output.copy_to_host())